In [1]:
from Pipeline import PreProcessingPipeline
from DataSplit import DataSplit

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline as SparkPipeline
from pyspark.ml import PipelineModel
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from Config import Config
import os



/home/roderickmajoor/anaconda3/envs/BD/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
preprocessing_pipeline = PreProcessingPipeline(DataSplit.TRAIN)

24/03/14 16:37:18 WARN Utils: Your hostname, roderick resolves to a loopback address: 127.0.1.1; using 192.168.178.29 instead (on interface enp3s0)
24/03/14 16:37:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/14 16:37:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = preprocessing_pipeline.run()

24/03/14 16:37:25 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
 Schema: id, product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
Expected: id but found: 
CSV file: file:///home/roderickmajoor/Desktop/Master/BD/BD-Project/data/train-3.csv
24/03/14 16:37:25 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
 Schema: id, product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
Expected: id but found: 
CSV file: file:///home/roderickmaj

Removed 0 duplicate rows.


In [4]:
df.show()

24/03/14 16:37:29 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
 Schema: id, product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
Expected: id but found: 
CSV file: file:///home/roderickmajoor/Desktop/Master/BD/BD-Project/data/train-6.csv


+---+----------+--------------+--------------------+----+-----------------+--------------------+--------------------+-----------+--------------+-------------------+-----+---------------+-------------+-------------+------+------+----------+----------+
| id|product_id|product_parent|       product_title|vine|verified_purchase|     review_headline|         review_body|review_date|marketplace_id|product_category_id|label|review_language|review_length|header_length|vine_Y|vine_N|verified_Y|verified_N|
+---+----------+--------------+--------------------+----+-----------------+--------------------+--------------------+-----------+--------------+-------------------+-----+---------------+-------------+-------------+------+------+----------+----------+
|  1|B00FAPF5U0|      46324555|    Candy Crush Saga|   N|                Y|brilliant but ann...|I took ages to si...| 2013-11-13|             1|                  1|False|             en|          160|           22|     0|     1|         1|        

In [5]:
spark = SparkSession.builder.appName("RandomForestExample").getOrCreate()


24/03/14 16:37:30 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [6]:
# List of your feature columns
featureColumns = ["review_length", "header_length", "vine_Y", "vine_N", "verified_Y", "verified_N"]

# Assemble the feature columns into a single feature vector
assembler = VectorAssembler(inputCols=featureColumns, outputCol="features")


In [7]:
# If "label" is not a numeric column, convert it to numeric using StringIndexer
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(df)


In [8]:
(trainingData, testData) = df.randomSplit([0.7, 0.3]) # before we submit, we want to train on all training data though.


In [9]:
# Configure the Random Forest model
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="features", numTrees=10)

# Chain indexers and forest in a Pipeline
preprocessing_pipeline = SparkPipeline(stages=[labelIndexer, assembler, rf])

# Train model
model = preprocessing_pipeline.fit(trainingData)
# save the model
# model.save("model")


24/03/14 16:37:32 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
 Schema: id, product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
Expected: id but found: 
CSV file: file:///home/roderickmajoor/Desktop/Master/BD/BD-Project/data/train-6.csv
24/03/14 16:37:35 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
 Schema: id, product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
Expected: id but found: 
CSV file: file:///home/roderickmaj

24/03/14 16:37:50 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
 Schema: id, product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
Expected: id but found: 
CSV file: file:///home/roderickmajoor/Desktop/Master/BD/BD-Project/data/train-2.csv
24/03/14 16:37:50 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
 Schema: id, product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
Expected: id but found: 
CSV file: file:///home/roderickmaj

In [10]:
if Config.MODEL_PATH in os.listdir():
    # delete model dir
    os.system(f"rm -r {Config.MODEL_PATH}")
model.save(Config.MODEL_PATH)

In [11]:
model = PipelineModel.load(Config.MODEL_PATH)

In [12]:
predictions = model.transform(testData)

# Select example rows to display
predictions.select("prediction", "indexedLabel", "features").show(5)


24/03/14 16:38:03 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
 Schema: id, product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
Expected: id but found: 
CSV file: file:///home/roderickmajoor/Desktop/Master/BD/BD-Project/data/train-6.csv


+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       0.0|         0.0|[160.0,22.0,0.0,1...|
|       0.0|         1.0|[307.0,17.0,0.0,1...|
|       0.0|         0.0|[148.0,0.0,0.0,1....|
|       0.0|         0.0|[81.0,10.0,0.0,1....|
|       0.0|         0.0|[193.0,16.0,0.0,1...|
+----------+------------+--------------------+
only showing top 5 rows



In [13]:
evaluator = BinaryClassificationEvaluator(labelCol="indexedLabel")
accuracy = evaluator.evaluate(predictions)
print(f"accuracy: {accuracy}")


24/03/14 16:38:06 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
 Schema: id, product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
Expected: id but found: 
CSV file: file:///home/roderickmajoor/Desktop/Master/BD/BD-Project/data/train-8.csv
24/03/14 16:38:06 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
 Schema: id, product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
Expected: id but found: 
CSV file: file:///home/roderickmaj

accuracy: 0.7360143799735283
